In [6]:
using DelimitedFiles 
using PyPlot
using Printf
import FlowFarm; const ff = FlowFarm

FlowFarm

# Round

## 12 directions

### Set Up

In [13]:
# include case information
include("./inputfiles/model_set_9_38turb_12dir_round_farm.jl");

### Run TIP Model

In [14]:
AEPTIP = ff.tip_get_aep(turbine_x,turbine_y,wind_data,cpdata, ctdata, r0=diam/2.0)

1.766993833025154e11

### Run FLOWFarm - Jensen TopHat Model

In [15]:
wakedeficitmodel = ff.JensenTopHat()
wakedeflectionmodel = ff.JiminezYawDeflection()
wakecombinationmodel = ff.SumOfSquaresLocalVelocitySuperposition()
localtimodel = ff.LocalTIModelNoLocalTI()
model_set = ff.WindFarmModelSet(wakedeficitmodel, wakedeflectionmodel, wakecombinationmodel, localtimodel)

AEPJTH = ff.calculate_aep(turbine_x, turbine_y, turbine_z, rotor_diameter,
                hub_height, turbine_yaw, ct_models, generator_efficiency, cut_in_speed,
                cut_out_speed, rated_speed, rated_power, windresource, power_models, model_set,
                rotor_sample_points_y=rotor_points_y,rotor_sample_points_z=rotor_points_z)

2.063563314949788e11

### Run FLOWFarm - Jensen Cosine Model

In [16]:
wakedeficitmodel = ff.JensenCosine()
wakedeflectionmodel = ff.JiminezYawDeflection()
wakecombinationmodel = ff.SumOfSquaresLocalVelocitySuperposition()
localtimodel = ff.LocalTIModelNoLocalTI()
model_set = ff.WindFarmModelSet(wakedeficitmodel, wakedeflectionmodel, wakecombinationmodel, localtimodel)

AEPJC = ff.calculate_aep(turbine_x, turbine_y, turbine_z, rotor_diameter,
                hub_height, turbine_yaw, ct_models, generator_efficiency, cut_in_speed,
                cut_out_speed, rated_speed, rated_power, windresource, power_models, model_set,
                rotor_sample_points_y=rotor_points_y,rotor_sample_points_z=rotor_points_z)

1.1294168535272508e11

### Run FLOWFarm - Bastankhah Model

In [17]:
wakedeficitmodel = ff.GaussYaw()
wakedeflectionmodel = ff.GaussYawDeflection()
wakecombinationmodel = ff.LinearLocalVelocitySuperposition()
localtimodel = ff.LocalTIModelMaxTI()
model_set = ff.WindFarmModelSet(wakedeficitmodel, wakedeflectionmodel, wakecombinationmodel, localtimodel)

AEPB = ff.calculate_aep(turbine_x, turbine_y, turbine_z, rotor_diameter,
                hub_height, turbine_yaw, ct_models, generator_efficiency, cut_in_speed,
                cut_out_speed, rated_speed, rated_power, windresource, power_models, model_set,
                rotor_sample_points_y=rotor_points_y,rotor_sample_points_z=rotor_points_z)

1.689974816296317e11

### Run FLOWFarm MultiZone Model

In [18]:
wakedeficitmodel = ff.Multizone()
wakedeflectionmodel = ff.JiminezYawDeflection()
wakecombinationmodel = ff.SumOfSquaresLocalVelocitySuperposition()
localtimodel = ff.LocalTIModelNoLocalTI()
model_set = ff.WindFarmModelSet(wakedeficitmodel, wakedeflectionmodel, wakecombinationmodel, localtimodel)

AEPM = ff.calculate_aep(turbine_x, turbine_y, turbine_z, rotor_diameter,
                hub_height, turbine_yaw, ct_models, generator_efficiency, cut_in_speed,
                cut_out_speed, rated_speed, rated_power, windresource, power_models, model_set,
                rotor_sample_points_y=rotor_points_y,rotor_sample_points_z=rotor_points_z)

2.1872586471228308e11

### Compare TIP vs TopHat

In [21]:
ErrorTIPvsTopHat = 100*(AEPTIP-AEPJTH)/AEPJTH
ErrorTIPvsCosine = 100*(AEPTIP-AEPJC)/AEPJC
ErrorTIPvsMultizone = 100*(AEPTIP-AEPM)/AEPM
ErrorTIPvsBastankhah = 100*(AEPTIP-AEPB)/AEPB
println("TIP AEP: ", AEPTIP)
println("Jensen Top Hat AEP:   ", AEPJTH)
println("Jensen Cosine AEP:   ", AEPJC)
println("Multi-Zone AEP:   ", AEPM)
println("Bastankhah AEP:   ", AEPB)
@printf("Error vs Jensen Top Hat:  %.3f%%\n", ErrorTIPvsTopHat)
@printf("Error vs Jensen Cosine:   %.3f%%\n", ErrorTIPvsCosine)
@printf("Error vs Multi-Zone:  %.3f%%\n", ErrorTIPvsMultizone)
@printf("Error vs Bastankhah:   %.3f%%\n", ErrorTIPvsBastankhah)

TIP AEP: 1.766993833025154e11
Jensen Top Hat AEP:   2.063563314949788e11
Jensen Cosine AEP:   1.1294168535272508e11
Multi-Zone AEP:   2.1872586471228308e11
Bastankhah AEP:   1.689974816296317e11
Error vs Jensen Top Hat:  -14.372%
Error vs Jensen Cosine:   56.452%
Error vs Multi-Zone:  -19.214%
Error vs Bastankhah:   4.557%


## 36 directions

### Set Up

In [34]:
# include case information
include("./inputfiles/model_set_9_38turb_36dir_round_farm.jl");

### Run TIP Model

In [35]:
AEPTIP = ff.tip_get_aep(turbine_x,turbine_y,wind_data,cpdata, ctdata,r0=diam/2.0)

5.4247163582650345e10

### Run FLOWFarm - Jensen TopHat Model

In [36]:
wakedeficitmodel = ff.JensenTopHat()
wakedeflectionmodel = ff.JiminezYawDeflection()
wakecombinationmodel = ff.SumOfSquaresLocalVelocitySuperposition()
localtimodel = ff.LocalTIModelNoLocalTI()
model_set = ff.WindFarmModelSet(wakedeficitmodel, wakedeflectionmodel, wakecombinationmodel, localtimodel)

AEPJTH = ff.calculate_aep(turbine_x, turbine_y, turbine_z, rotor_diameter,
                hub_height, turbine_yaw, ct_models, generator_efficiency, cut_in_speed,
                cut_out_speed, rated_speed, rated_power, windresource, power_models, model_set,
                rotor_sample_points_y=rotor_points_y,rotor_sample_points_z=rotor_points_z)

6.814580788138366e10

### Run FLOWFarm - Jensen Cosine Model

In [37]:
wakedeficitmodel = ff.JensenCosine()
wakedeflectionmodel = ff.JiminezYawDeflection()
wakecombinationmodel = ff.SumOfSquaresLocalVelocitySuperposition()
localtimodel = ff.LocalTIModelNoLocalTI()
model_set = ff.WindFarmModelSet(wakedeficitmodel, wakedeflectionmodel, wakecombinationmodel, localtimodel)

AEPJC = ff.calculate_aep(turbine_x, turbine_y, turbine_z, rotor_diameter,
                hub_height, turbine_yaw, ct_models, generator_efficiency, cut_in_speed,
                cut_out_speed, rated_speed, rated_power, windresource, power_models, model_set,
                rotor_sample_points_y=rotor_points_y,rotor_sample_points_z=rotor_points_z)

3.485464745806654e10

### Run FLOWFarm - Bastankhah Model

In [38]:
wakedeficitmodel = ff.GaussYaw()
wakedeflectionmodel = ff.GaussYawDeflection()
wakecombinationmodel = ff.LinearLocalVelocitySuperposition()
localtimodel = ff.LocalTIModelMaxTI()
model_set = ff.WindFarmModelSet(wakedeficitmodel, wakedeflectionmodel, wakecombinationmodel, localtimodel)

AEPB = ff.calculate_aep(turbine_x, turbine_y, turbine_z, rotor_diameter,
                hub_height, turbine_yaw, ct_models, generator_efficiency, cut_in_speed,
                cut_out_speed, rated_speed, rated_power, windresource, power_models, model_set,
                rotor_sample_points_y=rotor_points_y,rotor_sample_points_z=rotor_points_z)

5.211596064185484e10

### Run FLOWFarm MultiZone Model

In [39]:
wakedeficitmodel = ff.Multizone()
wakedeflectionmodel = ff.JiminezYawDeflection()
wakecombinationmodel = ff.SumOfSquaresLocalVelocitySuperposition()
localtimodel = ff.LocalTIModelNoLocalTI()
model_set = ff.WindFarmModelSet(wakedeficitmodel, wakedeflectionmodel, wakecombinationmodel, localtimodel)

AEPM = ff.calculate_aep(turbine_x, turbine_y, turbine_z, rotor_diameter,
                hub_height, turbine_yaw, ct_models, generator_efficiency, cut_in_speed,
                cut_out_speed, rated_speed, rated_power, windresource, power_models, model_set,
                rotor_sample_points_y=rotor_points_y,rotor_sample_points_z=rotor_points_z)

7.140181038618155e10

### Compare TIP vs TopHat

In [40]:
ErrorTIPvsTopHat = 100*(AEPTIP-AEPJTH)/AEPJTH
ErrorTIPvsCosine = 100*(AEPTIP-AEPJC)/AEPJC
ErrorTIPvsMultizone = 100*(AEPTIP-AEPM)/AEPM
ErrorTIPvsBastankhah = 100*(AEPTIP-AEPB)/AEPB
println("TIP AEP: ", AEPTIP)
println("Jensen Top Hat AEP:   ", AEPJTH)
println("Jensen Cosine AEP:   ", AEPJC)
println("Multi-Zone AEP:   ", AEPM)
println("Bastankhah AEP:   ", AEPB)
@printf("Error vs Jensen Top Hat:  %.2f%%\n", ErrorTIPvsTopHat)
@printf("Error vs Jensen Cosine:   %.2f%%\n", ErrorTIPvsCosine)
@printf("Error vs Multi-Zone:  %.2f%%\n", ErrorTIPvsMultizone)
@printf("Error vs Bastankhah:   %.2f%%\n", ErrorTIPvsBastankhah)

TIP AEP: 5.4247163582650345e10
Jensen Top Hat AEP:   6.814580788138366e10
Jensen Cosine AEP:   3.485464745806654e10
Multi-Zone AEP:   7.140181038618155e10
Bastankhah AEP:   5.211596064185484e10
Error vs Jensen Top Hat:  -20.40%
Error vs Jensen Cosine:   55.64%
Error vs Multi-Zone:  -24.03%
Error vs Bastankhah:   4.09%
